In [1]:
import sys
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
sys.path.append("E:/university/Year 5 Spring/FYT/code/multi_modal_ser")
from utils.dataset import MMSERDataset
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
mmser_ds = torch.load("E:/datasets/preprocessed/dataset/mmser_ds.pt")

### Audio Classification

In [3]:
import torch
from transformers import AutoFeatureExtractor, WhisperForAudioClassification

model = WhisperForAudioClassification.from_pretrained("openai/whisper-large").to(device)
input_features = mmser_ds[78:79]["audio"]

Some weights of WhisperForAudioClassification were not initialized from the model checkpoint at openai/whisper-large and are newly initialized: ['model.projector.weight', 'model.projector.bias', 'model.classifier.weight', 'model.classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
with torch.no_grad():
    logits = model(input_features).logits

predicted_class_ids = torch.argmax(logits, axis=1).item()
predicted_label = model.config.id2label[predicted_class_ids]
predicted_label

'LABEL_1'

### Build Model

In [5]:
print("Emotions: ", mmser_ds.df_["emotion"].unique())
model.config.num_labels = mmser_ds.df_["emotion"].nunique()
model.classifier = nn.Linear(model.projector.out_features, model.config.num_labels)
# model

Emotions:  ['neu' 'fru' 'ang' 'sad' 'hap' 'sur' 'exc' 'oth' 'fea' 'dis']


### Train Model

In [6]:
train_size = int(len(mmser_ds)*0.7)
val_size = int(len(mmser_ds)*0.2)
test_size = len(mmser_ds)-int(len(mmser_ds)*0.7)-int(len(mmser_ds)*0.2)

train_set, val_set = torch.utils.data.random_split(mmser_ds, [train_size, val_size+test_size])
val_set, test_set = torch.utils.data.random_split(val_set, [val_size, test_size])

In [7]:
from transformers import TrainingArguments, Trainer

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

training_args = TrainingArguments(output_dir="test_trainer")
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_set,
    eval_dataset=val_set,
    compute_metrics=compute_metrics,
)

In [8]:
train_loader = DataLoader(train_set, batch_size=4)
val_loader = DataLoader(val_set, batch_size=4)
test_loader = DataLoader(test_set, batch_size=4)

In [27]:
from torch import nn
from transformers import Trainer
from transformers import TrainingArguments
import datetime

training_args = TrainingArguments(output_dir=datetime.datetime.now().date().strftime(format="%Y-%m-%d"))

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels").to(device)
        outputs = model(input_features=inputs["audio"].to(device))
        logits = outputs.get("logits")
        # compute custom loss (suppose one has 3 labels with different weights)
        loss_fct = nn.CrossEntropyLoss() # weight=torch.tensor([1.0, 2.0, 3.0], device=model.device))
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

In [28]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [29]:
training_args.logging_steps = 5
training_args.remove_unused_columns=False
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_set,
    eval_dataset=val_set,
    compute_metrics=compute_metrics
)

In [30]:
# https://huggingface.co/blog/fine-tune-whisper

In [31]:
# training_args

In [ ]:
trainer.train()